In [8]:
from datasets import load_metric
wer_metric = load_metric("wer")
cer_metric = load_metric("cer")

import pandas as pd
import os
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor, Wav2Vec2CTCTokenizer
import soundfile as sf
import os
# Use old or new vocab? 
#os.system("cp vocab_300_with_numbers.json vocab.json")



tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(
        "/home/peterr/macocu/task8/", unk_token="[UNK]", 
        pad_token="[PAD]", word_delimiter_token=" ")
feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(
    feature_extractor=feature_extractor, tokenizer=tokenizer)
model = Wav2Vec2ForCTC.from_pretrained("/home/peterr/macocu/task8/31_normalised_220k/checkpoint-6500").cuda()

def transcribe(path):
    speech, sample_rate = sf.read(path)
    input_values = processor(speech, sampling_rate=sample_rate, return_tensors="pt").input_values.cuda()

    logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0]).lower()
    return transcription


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
import datasets
from datasets import load_dataset, load_metric, Audio, load_from_disk
test = load_from_disk("test_mapped")

In [10]:
speech, sample_rate = test[0]["input_values"], 16000
input_values = processor(speech, sampling_rate=sample_rate, return_tensors="pt").input_values.cuda()

logits = model(input_values).logits

predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.decode(predicted_ids[0]).lower()
transcription

'zahtjevima europske komisije u odnosu na direkdirektivu europskog parlamenta i vijeća od 5 srpnja 206 godine zahvaljujem se na podršci svima koji su raspravljali i na samomekraju vezano uz ratifikaciju istambolske konvencije'

In [11]:
labs = test[0]["labels"]
processor.decode(labs).lower()

'zahtjevima europske komisije u odnosu na direktivu europskog parlamenta i vijeća od 5. srpnja 206. godine zahvaljujem se na podršci svima koji su raspravljali i na samome kraju vezano uz ratifikaciju istambulske konvencije'

In [17]:
def decode_labels(labels):
    return processor.decode(labels).lower()
def transcribe(inputs):
    speech = inputs
    input_values = processor(speech, sampling_rate=16000, return_tensors="pt").input_values.cuda()

    logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0]).lower()
    return transcription

transcriptions = list(map(transcribe, test["input_values"]))
labels = list(map(decode_labels, test["labels"]))

In [21]:
transcriptions[2],labels[2]

('transparentnosti cijelog sustava javne nabave tijekom 2018. godina držav na ko misija ob obilježilaje 1.u odišnjih surada koja je obilježena međunarnom konferencijom koja je okupila renomirane stručnjeke na području javne nabave iz rh tinozemstva kao i predstavnike akadems',
 'transparentnosti cijelog sustava javne nabave tijekom 2018g državna komisija obilježila je 15tu godišnjicu rada koja je obilježena međunarodnom konferencijom koja je okupila renomirane stručnjake na području javne nabave iz rh te inozemstva kao i predstavnike akademske')

In [22]:
import pandas as pd
pd.DataFrame(data={
    "machine_generated_transcripts": transcriptions,
    "original_transcript": labels
}).to_csv("34_model31_220k_performance.csv", index=False)